In [48]:
import pandas as pd

In [49]:
weapons_data = pd.read_csv('weapons.csv')

In [55]:
def table_to_text(df, id_col, drop_cols):
    df = df.drop(columns=drop_cols)
    df_melted = df.melt(id_vars=id_col)
    assert len(df_melted) == len(df) * (len(df.columns) - 1)
    df_melted['value'] = df_melted['value'].astype(str)
    text_df = pd.DataFrame(columns=['text'])
    text_df['text'] = 'The ' + df_melted['variable'] + ' of ' + df_melted['name'] + ' is ' + df_melted['value']
    return text_df


In [56]:
# weapons.csv
weapons_txt = table_to_text(weapons_data, 'name', ['id', 'weapon_id', 'image'])
weapons_txt.to_csv('weapon_txt.csv', index=False)


In [52]:
print(weapons_txt.head())

                                            text
0            The weight of Dueling Shield is 9.0
1  The weight of Carian Thrusting Shield is 10.5
2              The weight of Dryleaf Arts is 1.0
3           The weight of Dane's Footwork is 1.0
4        The weight of Smithscript Dagger is 1.5
